In [1]:
import os 
import sys 
import openai
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
print(os.getcwd())

c:\Users\TempAccess\Documents\Dhruv\RAG\simple_rag


In [3]:
os.chdir(r"C:\Users\TempAccess\Documents\Dhruv\RAG")
print(os.getcwd())

C:\Users\TempAccess\Documents\Dhruv\RAG


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
from helper_functions import (EmbeddingProvider,
                              retrieve_context_per_question,
                              replace_t_with_space,
                              get_langchain_embedding_provider,
                              show_context)

from evaluation.evalute_rag import evaluate_rag

from langchain_community.vectorstores import FAISS


In [6]:
import os
os.makedirs('data', exist_ok=True)

In [7]:
path = "data/Understanding_Climate_Change.pdf"

# Lets Encode the PDF.

In [8]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.

    """

    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split Documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len
        )

    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # create embeddings
    embeddings = get_langchain_embedding_provider(EmbeddingProvider.OPENAI)

    vector_store = FAISS.from_documents(cleaned_texts, embeddings)

    return vector_store

In [9]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

# Create retriver

In [10]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

# Test retriever

In [11]:
test_query = "What is the main cause of climate change?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)


Context 1:
Chapter 2: Causes of Climate Change 
Greenhouse Gases 
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate. 
Fossil Fuels 
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today. 
Coal


Context 2:
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch,